In [1]:
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.metrics import RMSE
from cornac.models import  ItemKNN, UserKNN, NMF, BPR, MF,VAECF, NeuMF,PMF, WMF
# from MF import MF
import pandas as pd
import numpy as np
import cornac
import math
# import seaborn as sns
# import matplotlib.pyplot as plt

/Users/tahsinalamgirkheya/anaconda3/envs/re-ranking/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# rating_data_pd = pd.read_csv(
#     "../data/ml-100K/indexed_interactions.csv",
#     sep="\t",
#     header=0,
#     names=["itemID","_","userID", "Rating", "Gender"],
# )
# # itemID	actual_category	userID	Rating	Gender

# rating_data = rating_data_pd.to_numpy()
# rating_data.__len__()
# rating_data_pd

reader = Reader()
rating_data_pd = pd.read_csv(
    "../data/yelp-100K/indexed_interactions.csv",
    sep="\t",
    header=0,
    names=[ "itemID","Category", "userID","Rating", "Gender"],
)
rating_data = rating_data_pd[["userID", "itemID","Rating"]].to_numpy()
rating_data.__len__()
rating_data_pd


,itemID,Category,userID,Rating,Gender
0,0,Food & Restaurants|Drinks & Spirits,0,5,F
1,0,Food & Restaurants|Drinks & Spirits,1,4,F
2,0,Food & Restaurants|Drinks & Spirits,2,5,F
3,0,Food & Restaurants|Drinks & Spirits,3,3,M
4,0,Food & Restaurants|Drinks & Spirits,4,3,F
...,...,...,...,...,...
97986,1271,Food & Restaurants,1060,3,F
97987,1271,Food & Restaurants,161,2,M
97988,1271,Food & Restaurants,25,3,F
97989,1271,Food & Restaurants,280,4,F


In [3]:

restaurants = pd.read_csv("../data/yelp-100K/i_id_mapping.csv",sep="\t",
    header=0,
    names=[ "item_id","Category","itemID"])
restaurants=restaurants.sort_values(by="itemID")

unique_categories = [
    "Active Life & Fitness",
    "Arts & Entertainment",
    "Automotive",
    "Bars & Nightlife",
    "Coffee,Tea & Desserts",
    "Drinks & Spirits",
    "Education & Learning",
    "Event Services",
    "Family & Kids",
    "Food & Restaurants",
    "Health & Beauty",
    "Home & Garden",
    "Miscellaneous",
    "Outdoor Activities",
    "Public Services & Community",
    "Shopping & Fashion",
    "Specialty Food & Groceries",
    "Sports & Recreation",
    "Technology & Electronics",
    "Travel & Transportation",
    "Asian",
]
for c in unique_categories:
    restaurants[c] = 0
for index, row in restaurants.iterrows():
    cats = row["Category"].split("|")
    for cat in cats:
        restaurants.at[index, cat] = 1

cat = restaurants[unique_categories]
# cat[:1]
item_features_numpy = cat.to_numpy()
item_features = {
    str(item_id): {"category_" + str(idx): value for idx, value in enumerate(row)}
    for item_id, row in enumerate(item_features_numpy)
}
# ids = list(range(0, 3416))

In [4]:
restaurants

,item_id,Category,itemID,Active Life & Fitness,Arts & Entertainment,Automotive,Bars & Nightlife,"Coffee,Tea & Desserts",Drinks & Spirits,Education & Learning,...,Home & Garden,Miscellaneous,Outdoor Activities,Public Services & Community,Shopping & Fashion,Specialty Food & Groceries,Sports & Recreation,Technology & Electronics,Travel & Transportation,Asian
0,34,Food & Restaurants|Drinks & Spirits,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,250,Food & Restaurants|Drinks & Spirits,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,759,Asian|Food & Restaurants,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1085,Food & Restaurants,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1122,Travel & Transportation,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,188255,Food & Restaurants,1267,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1268,188310,Food & Restaurants,1268,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1269,188449,Food & Restaurants|Bars & Nightlife,1269,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,188454,Food & Restaurants,1270,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:

users = pd.read_csv("../data/yelp-100K/u_id_mapping.csv", sep="\t",header=0,
    names=[ "user_id","Gender","userID"])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
users = users.sort_values(by="userID")
users = users[["Gender", "userID"]]

user_features_numpy = users.to_numpy()
print(user_features_numpy.shape)
# print(item_features_numpy.shape)

(1316, 2)


In [6]:
dataset = rating_data
dataset

array([[   0,    0,    5],
       [   1,    0,    4],
       [   2,    0,    5],
       ...,
       [  25, 1271,    3],
       [ 280, 1271,    4],
       [ 955, 1271,    5]])

In [7]:
users

,Gender,userID
0,1,0
1,1,1
2,1,2
3,0,3
4,1,4
...,...,...
1311,0,1311
1312,1,1312
1313,1,1313
1314,0,1314


In [32]:
# ratio_split = StratifiedSplit(
#     data=dataset,
#     test_size=0.2,
#     rating_threshold=0.0,
#     seed=123,
#     verbose=True,
#     chrono=True,
# )
ratio_split = StratifiedSplit(
    data=dataset,
    test_size=0.2,
    rating_threshold=0,
    # val_size=0.1,
    seed=123,
    verbose=True,fmt="UIR"
)

hr_10 = cornac.metrics.HitRatio(k=20)
ndcg_10 = cornac.metrics.NDCG(k=20)
recall_10 = cornac.metrics.Recall(k=20)
prec_10 = cornac.metrics.Precision(k=20)
auc = cornac.metrics.AUC()
map = cornac.metrics.MAP()

epochs = [20, 40, 60, 80, 100]
models = []
# class cornac.models.vaecf.recom_vaecf.
# VAECF(name='VAECF', k=10, autoencoder_structure=[20], act_fn='tanh',
# likelihood='mult', n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, trainable=True, verbose=False, seed=None, use_gpu=False)[source]
# for i in range(len(epochs)):
#     models.append(
#     #    PMF(k=5, max_iter=epochs[i], learning_rate=0.001, gamma=0.9, lambda_reg=0.001, name=f'PMF{epochs[i]}', variant='linear', trainable=True, verbose=True, init_params=None, seed=123)
#        MF(
#             name=f"MF{epochs[i]}",
#             k=10,
#             backend="cpu",
#             optimizer="adam",
#             max_iter=epochs[i],
#             learning_rate=0.01,
#             batch_size=256,
#             lambda_reg=0.02,
#             dropout=0.0,
#             use_bias=True,
#             early_stop=False,
#             num_threads=0,
#             trainable=True,
#             verbose=False,
#             init_params=None,
#             seed=123,
#         )
#   )
# model = WMF(name=f'WMF{100}', k=200, lambda_u=0.01, lambda_v=0.01, a=1, b=0.01, learning_rate=0.001, batch_size=128, max_iter=100, trainable=True, verbose=True, init_params=None, seed=123)

model = MF(
            name=f"MF{40}",
            k=10,
            backend="cpu",
            optimizer="adam",
            max_iter=40,
            learning_rate=0.01,
            batch_size=256,
            lambda_reg=0.02,
            dropout=0.0,
            use_bias=True,
            early_stop=False,
            num_threads=0,
            trainable=True,
            verbose=False,
            init_params=None,
            seed=123,
        )

# model =  NeuMF(
#            name=f"NeuMFe={80}",
#             num_factors=8,
#             layers=( 32, 16, 8),
#             act_fn="sigmoid",
#             reg=0.0,
#             num_epochs=80,
#             batch_size=256,
#             num_neg=3,
#             lr=0.001,
#             learner="adam",
#             backend="tensorflow",
#             early_stopping=None,
#             trainable=True,
#             verbose=True,
#             seed=123,
#         )
# # model=VAECF(k=10, autoencoder_structure=[20], name="vae100",act_fn="tanh", likelihood="mult", n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, seed=123, verbose=True)

models = [model]
cornac.Experiment(
    ratio_split, models=models, metrics=[hr_10, ndcg_10, recall_10, auc, prec_10, map]
).run()

rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 1316
Number of items = 1272
Number of ratings = 77877
Max rating = 5.0
Min rating = 1.0
Global mean = 3.9
---
Test data:
Number of users = 1316
Number of items = 1272
Number of ratings = 20114
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1316
Total items = 1272

[MF40] Training started!

[MF40] Evaluation started!


Ranking: 100%|██████████| 1316/1316 [00:00<00:00, 2176.08it/s]


TEST:
...
     |    AUC | HitRatio@20 |    MAP | NDCG@20 | Precision@20 | Recall@20 | Train (s) | Test (s)
---- + ------ + ----------- + ------ + ------- + ------------ + --------- + --------- + --------
MF40 | 0.5280 |      0.2948 | 0.0215 |  0.0234 |       0.0193 |    0.0236 |    0.0655 |   0.6100



In [15]:
user_ids = users["userID"].to_numpy()
item_ids = restaurants["itemID"].to_numpy()
item_ids

array([   0,    1,    2, ..., 1269, 1270, 1271])

In [10]:
users

,Gender,userID
0,1,0
1,1,1
2,1,2
3,0,3
4,1,4
...,...,...
1311,0,1311
1312,1,1312
1313,1,1313
1314,0,1314


In [11]:
user_ids.__len__()

1316

In [33]:
# get the top_k ratings for all users:
top_k = 100
reco_matrix = np.zeros((len(models), len(user_ids), top_k), dtype=int)
reco_matrix_mapped_items = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=int
)
reco_matrix_mapped_scores = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=float
)
reco_matrix_all = np.zeros((len(models), len(user_ids), len(item_ids)), dtype=int)


for u in user_ids:
    for i in range(len(models)):
        reco_items = models[i].recommend(u)
        items_mapped, mapped_scores = models[i].rank(
            user_idx=u, item_indices=list(item_ids)
        )
        reco_matrix_mapped_items[i][u] = items_mapped
        reco_matrix_mapped_scores[i][u] = mapped_scores
        reco_matrix_all[i][u] = reco_items
        reco_matrix[i][u] = reco_items[:top_k]

        # print(reco_matrix[0][3])

In [36]:
reco_matrix[0][1]

array([1213,  775,   11,  873,  320, 1204,  762,  179, 1106,   69,  237,
        727,  824,  396,   13,  757, 1217,  267, 1032,  698,  736,  107,
       1170, 1060,  915,   55,  497,  662, 1063,  321,  811, 1227,  306,
        642,   97,   48,  931,  773, 1054, 1235,  524,  786,  394, 1080,
        243,  240,  549,  880, 1238,  836,  990, 1210,  389,  984,  542,
        559, 1013,  743,  142,  796, 1105,  471,   72,  490,   61,  291,
        443,  185, 1104,  844,  324,  515,   60,  164,  903,  458,  447,
        152,  176,  999, 1153,  751, 1242,   37, 1188, 1239,  327, 1070,
        201,  531,  528,  808,  676,  205,  110,  397, 1194,  356,  401,
        155])

In [37]:
reco_matrix[0][2]

array([1213,  458,  179,   11,  873,  320,  396,   13,  542,   69,  321,
        757,  824,  984, 1217,  775,  497,  642,  853, 1170,  267,  762,
        727, 1204,  662,  698, 1226, 1060,  736,  751,   97, 1110,  181,
       1106,   20,  524,  915,  949,  836,  113,  883,  394,  404, 1100,
        306, 1188,  786,   44,  515, 1013,  447, 1235, 1219, 1045,  880,
         55, 1095,  854,  443,  961, 1194,  476,  990, 1129,  985,  389,
       1210, 1063,  205,  146,  598, 1080,  243,  917,  330,  257,  237,
        773,  623, 1192, 1161,  848,  673,  820,  552,   72,  811, 1105,
        807, 1241,  525,   40,  138,  198, 1073,  726, 1104, 1242,  510,
        635])

In [17]:
test_set_data = pd.DataFrame(ratio_split.test_set.uir_tuple).transpose()
test_set_data.columns = ["uid", "iid", "rating"]
test_set_data = test_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

test_set_data["uid"] = test_set_data["uid"].map(r_global_uid_map)
test_set_data["iid"] = test_set_data["iid"].map(r_global_iid_map)
test_set_data

,uid,iid,rating
0,0,407,1
1,0,505,5
2,0,631,5
3,0,1032,1
4,0,266,1
...,...,...,...
20109,1315,690,3
20110,1315,214,4
20111,1315,1223,4
20112,1315,997,4


In [24]:
train_set_data = pd.DataFrame(ratio_split.train_set.uir_tuple).transpose()
train_set_data.columns = ["uid", "iid", "rating"]
train_set_data = train_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

train_set_data["uid"] = train_set_data["uid"].map(r_global_uid_map)
train_set_data["iid"] = train_set_data["iid"].map(r_global_iid_map)

In [26]:
train_set_data

,uid,iid,rating
0,0,25,4
1,0,900,5
2,0,774,2
3,0,468,5
4,0,529,2
...,...,...,...
77872,1315,666,4
77873,1315,839,4
77874,1315,348,4
77875,1315,824,4


In [25]:
test_set_data.to_csv("testing_set_seed123_yelp100k.csv", index=False, header=False)


In [27]:
train_set_data.to_csv("training_set_seed123_yelp100k.csv", index=False, header=False)


In [45]:
np.save("reco_matrix_mf_yelp100k_100",reco_matrix)


In [1]:
reco_matrix[0].shape

NameError: name 'reco_matrix' is not defined

In [40]:
from collections import OrderedDict
# uid_map (OrderDict, required) – The dictionary containing mapping from user original ids to mapped integer indices.

# iid_map (OrderDict, required) – The dictionary containing mapping from item original ids to mapped integer indices.
sorted_by_values = OrderedDict(sorted(models[0].iid_map.items(), key=lambda item: item[1]))
keys_sorted_by_values = list(sorted_by_values.keys())

reco_items_scores_all = [OrderedDict() for _ in range(len(user_ids))]

for u in user_ids:
    actual_index_u = u
    mapped_index_u = models[0].uid_map[actual_index_u]
    # print(f"u {u} mapped u {mapped_index_u}")
    
    # for i in item_ids:
    mapped_scores = reco_matrix_mapped_scores[0][mapped_index_u]
    ordered_dict = OrderedDict(zip(keys_sorted_by_values, mapped_scores))
    reco_items_scores_all[actual_index_u] = ordered_dict
    
    
    

In [41]:
reco_items_scores_all[0]

OrderedDict([(25, 3.326870918273926),
             (900, 5.021640777587891),
             (774, 1.394498586654663),
             (468, 4.536920547485352),
             (529, 2.341641902923584),
             (218, 3.9438376426696777),
             (996, 3.4856982231140137),
             (153, 3.3631718158721924),
             (855, 3.1620702743530273),
             (696, 3.946038007736206),
             (93, 2.857102632522583),
             (303, 4.81065559387207),
             (185, 3.46885347366333),
             (37, 2.798424005508423),
             (33, 3.6364738941192627),
             (0, 3.6093859672546387),
             (60, 3.3482935428619385),
             (628, 3.260652780532837),
             (1130, 1.9637771844863892),
             (1165, 4.420836925506592),
             (455, 3.6403560638427734),
             (292, 2.0471842288970947),
             (635, 4.473366737365723),
             (1054, 4.481782913208008),
             (1156, 2.0365562438964844),
             (176, 

In [17]:
models[0].uid_map.items()

odict_items([(101, 0), (845, 1), (705, 2), (20, 3), (635, 4), (593, 5), (153, 6), (37, 7), (915, 8), (522, 9), (810, 10), (18, 11), (34, 12), (809, 13), (239, 14), (749, 15), (889, 16), (928, 17), (542, 18), (501, 19), (342, 20), (175, 21), (116, 22), (144, 23), (618, 24), (118, 25), (860, 26), (335, 27), (263, 28), (825, 29), (405, 30), (942, 31), (867, 32), (286, 33), (393, 34), (224, 35), (146, 36), (792, 37), (416, 38), (652, 39), (147, 40), (368, 41), (773, 42), (727, 43), (194, 44), (246, 45), (75, 46), (446, 47), (76, 48), (461, 49), (793, 50), (912, 51), (110, 52), (504, 53), (84, 54), (627, 55), (621, 56), (42, 57), (594, 58), (728, 59), (301, 60), (214, 61), (668, 62), (937, 63), (484, 64), (274, 65), (685, 66), (526, 67), (776, 68), (121, 69), (139, 70), (108, 71), (679, 72), (207, 73), (111, 74), (65, 75), (587, 76), (143, 77), (322, 78), (392, 79), (770, 80), (299, 81), (838, 82), (690, 83), (630, 84), (27, 85), (829, 86), (11, 87), (676, 88), (26, 89), (380, 90), (490, 91

In [18]:
models[0].iid_map.items()

odict_items([(177, 0), (289, 1), (153, 2), (361, 3), (321, 4), (723, 5), (983, 6), (60, 7), (949, 8), (140, 9), (113, 10), (136, 11), (1024, 12), (311, 13), (347, 14), (240, 15), (111, 16), (52, 17), (31, 18), (297, 19), (320, 20), (88, 21), (367, 22), (489, 23), (490, 24), (568, 25), (355, 26), (985, 27), (68, 28), (402, 29), (95, 30), (710, 31), (203, 32), (175, 33), (426, 34), (161, 35), (722, 36), (488, 37), (405, 38), (445, 39), (841, 40), (771, 41), (698, 42), (450, 43), (415, 44), (83, 45), (302, 46), (11, 47), (364, 48), (388, 49), (124, 50), (336, 51), (955, 52), (1008, 53), (265, 54), (1170, 55), (268, 56), (637, 57), (188, 58), (298, 59), (41, 60), (1037, 61), (1082, 62), (529, 63), (165, 64), (198, 65), (847, 66), (158, 67), (499, 68), (305, 69), (500, 70), (73, 71), (267, 72), (761, 73), (1066, 74), (1110, 75), (350, 76), (832, 77), (687, 78), (913, 79), (306, 80), (514, 81), (810, 82), (1010, 83), (1124, 84), (1122, 85), (412, 86), (644, 87), (562, 88), (1138, 89), (196, 

In [19]:
i,s=models[0].rank(98, item_ids)
print(s.__len__())
np.argsort(s)[-10:][::-1]

1348


array([303,  17,  25, 193, 136,   9, 466, 380, 147, 421])

In [ ]:
matching_keys = [key for key, value in models[0].iid_map.items() if value == 35]
matching_keys


In [ ]:
reco_items_scores_all.__len__()

In [44]:
import pickle
with open('score_dicts_mf_yelp100k.pkl', 'wb') as file:
    pickle.dump(reco_items_scores_all, file)

print("List of OrderedDicts saved to 'score_dicts.pkl'.")


List of OrderedDicts saved to 'score_dicts.pkl'.


In [ ]:
reco_matrix[0][0]

In [ ]:
reco_items_scores_all[98]

In [ ]:
x = OrderedDict(sorted(reco_items_scores_all[772].items(), key=lambda item: item[1], reverse=True))
x
